In [1]:
using QPControl
using RigidBodyDynamics
using RigidBodyDynamics.Contact
using StaticArrays
using SimpleQP
using Rotations
using Gurobi

In [27]:
function add_base_joint!(mechanism, label, direction)
    frame = CartesianFrame3D("base_$(label)_dummy")
    inertia = SpatialInertia(frame, SDiagonal(0., 0, 0), SVector(0., 0, 0), 0.0)
    body = RigidBody(inertia)
    joint = Joint("base_$(label)", Prismatic(direction))
    effort_bounds(joint) .= RigidBodyDynamics.Bounds(0, 0)
    parent = last(bodies(mechanism))
    attach!(mechanism, parent, body, joint)
end

function three_dof_point_mass()
    world = RigidBody{Float64}("world")
    mechanism = Mechanism(world, gravity=SVector(0, 0, -9.81))
    add_base_joint!(mechanism, "x", SVector(1., 0, 0))
    add_base_joint!(mechanism, "y", SVector(0., 1, 0))
    add_base_joint!(mechanism, "z", SVector(0., 0, 1))

    frame = CartesianFrame3D("core")
    inertia = SpatialInertia(frame, SDiagonal(0.01, 0.01, 0.01), SVector(0., 0, 0), 1.0)
    body = RigidBody(inertia)
    joint = Joint("core_to_base", Fixed{Float64}())
    parent = last(bodies(mechanism))
    joint_pose = Transform3D(frame_before(joint), default_frame(parent), RotXYZ(randn(), randn(), randn()))
    attach!(mechanism, parent, body, joint)
    mechanism
end

function add_floor!(mechanism, orientation, μ, contacting_body)
    world = root_body(mechanism)
    floor = HalfSpace3D(Point3D(default_frame(world), 0., 0, 0), FreeVector3D(default_frame(world), orientation * SVector(0., 0, 1)))
    add_environment_primitive!(mechanism, floor)

    contactmodel = SoftContactModel(hunt_crossley_hertz(k = 500e3), 
        ViscoelasticCoulombModel(μ, 20e3, 100.))
    add_contact_point!(contacting_body, Contact.ContactPoint(Point3D(default_frame(contacting_body), 0., 0, 0), contactmodel))
    floor
end

function mpc_lcp_feasibility_problem(mechanism, optimizer, floor, Δt)
    mpc = QPControl.MPCController{QPControl.ContactPoint{8}}(mechanism, optimizer)
    stage = QPControl.addstage!(mpc, 0.01)
    for body in bodies(mechanism)
        for point in RigidBodyDynamics.contact_points(body)
            position = location(point)
            μ = point.model.friction.μ
            contact = addcontact!(mpc, stage, position, floor, μ, QPControl.LCPContact())
            contact.maxnormalforce[] = 1e6
        end
    end
    mpc
end

function simulate_lcp!(state::MechanismState, mpc::MPCController, t_final=1.0)
    # Simulate the system by repeatedly running the MPC controller
    # and then updating the state based on the expected state
    # at the end of the first MPC stage.
    ts = Float64[]
    qs = Vector{Float64}[]
    vs = Vector{Float64}[]
    τ = similar(velocity(state))
    for t in 0 : (mpc.stages[1].Δt) : t_final
        push!(ts, t)
        push!(qs, configuration(state))
        push!(vs, velocity(state))
        mpc(τ, t, state)
        # Assume the dynamics behave exactly as modeled in the optimization
        set_configuration!(state, SimpleQP.value.(mpc.qpmodel, mpc.stages[1].q))
        set_velocity!(state, SimpleQP.value.(mpc.qpmodel, mpc.stages[1].v))
    end
    ts, qs, vs
end

simulate_lcp! (generic function with 2 methods)

In [21]:
using MeshCatMechanisms
using MeshCat

In [4]:
vis = Visualizer()
open(vis)

In [28]:
optimizer = GurobiOptimizer(OutputFlag=0)

mechanism = three_dof_point_mass()
μ = 0.1
floor = add_floor!(mechanism, RotX(0.0), μ, findbody(mechanism, "core"))
delete!(vis)
mvis = MechanismVisualizer(mechanism, Skeleton(), vis)

# Create an MPC "controller", which we will use as a simple
# one-step LCP simulator
Δt = 0.003
mpc = mpc_lcp_feasibility_problem(mechanism, optimizer, floor, Δt);

In [29]:
state = MechanismState(mechanism)
set_configuration!(state, [0., 0, 0.5])
set_velocity!(state, [-2.0, 0, 0])
ts, qs, vs = RigidBodyDynamics.simulate(state, 5.0, mpc; Δt=Δt)
setanimation!(mvis, ts, qs)

true

Academic license - for non-commercial use only


In [19]:
state = MechanismState(mechanism)
set_configuration!(state, [0., 0, 0.5])
set_velocity!(state, [-2.0, 0, 0])
ts, qs, vs = simulate_lcp!(state, mpc)
setanimation!(mvis, ts, qs)

true